In [1]:
from common import *
from numba import jit

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


### high level redesign

In [2]:
class TheAlg():
    def __init__(a, X, Y, W, s):
        a.s = s
        a._alg_evaluate = {2: _alg_evaluate_2, 3: _alg_evaluate_3}[s.type]
        a.k = X.columns.get_level_values(0).nunique() # features
        a.m = X.columns.get_level_values(1).nunique() # stocks/groups
        a.columns = X.columns
        a._def_matrices(X, Y, W, s) # Y not demeaned yet to get the no mean baseline score
        a._setup()
        
    def _setup(a):
        a.scores = {}
        a.scores[None] = a._calc_baseline()
        a.meandata = (a.Y.tc * a.W.tc).sum(axis=0) / _replace_0_1(a.W.tc.sum(axis=0))
        for sp in a.Y: # loop to demean Y
            a.Y[sp] -= a.meandata
        a.scores[()] = a._calc_baseline()
        a.models = {None: 0, (): a.meandata}
        
    def _def_matrices(a, X, Y, W, s):
        a.X, a.Y, a.W = O(), O(), O()
        for aZ, Z in zip([a.X, a.Y, a.W], [X, Y, W]):
            aZ.tc = Z[s.tc].values
            aZ.d0 = np.stack([Z[tr].values for tr, cv in s.trcv])
            aZ.d1 = np.stack([Z[cv].values for tr, cv in s.trcv])
            aZ.ho = Z[s.ho].values
            aZ.fn = Z[s.fn].values
        
    def _calc_baseline(a):
        return (sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(a.Y.tc, a.W.tc)) * a.s.repeat,
                sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(a.Y.ho, a.W.ho)))

    def alg1(a, start=(), branch=1, branch_at_depth=None, max_depth=4, print=print, dot=1, numdot=10):
        a._alg1 = O(branch=branch, branch_at_depth=branch_at_depth, max_depth=max_depth, print=print, dot=dot, numdot=numdot)
        a._alg1_recur(start, depth=0)
        
    def _alg1_recur(a, feats, depth):
        thefeats, o, print = feats, a._alg1, a._alg1.print
        print(a.scores[feats][0] / a.scores[None][0], a.scores[feats][1] / a.scores[None][1], len(feats))
        print('[' + ', '.join(str(a.columns[a.m * f][0]) for f in feats) + '] ' + str(feats))
        print(flush=True)
        if depth >= o.max_depth:
            return
        print('<{}>'.format(a.k - len(feats)), end=' ')
        heap = []
        for i, f in enumerate(set(range(a.k)) - set(thefeats)):
            print('.' if i // o.dot % o.numdot else i, end='') if i % o.dot == 0 else None
            newfeats = tuple(sorted(thefeats + (f,)))
            if newfeats not in a.scores:
                newscore, newmodel = a._alg_evaluate(a.X.d0, a.Y.d0, a.W.d0,
                                                     a.X.d1, a.Y.d1, a.W.d1,
                                                     a.X.tc, a.Y.tc, a.W.tc,
                                                     a.X.ho, a.Y.ho, a.W.ho, a.m, a.k, np.array(newfeats, dtype=np.int64))
                a.scores[newfeats] = newscore
                a.models[newfeats] = newmodel
            heap.append((a.scores[newfeats][0], newfeats))
        print(flush=True)
        heap.sort()
        for score0, feats in heap[:o.branch_at_depth(depth) if o.branch_at_depth else o.branch]:
            a._alg1_recur(feats, depth + 1)

In [3]:
def _replace_0_1(a):
    a[a == 0] = 1
    return a

In [4]:
@jit(nopython=True)
def _alg_evaluate_3(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, k, subk):
    tc = np.empty(m, dtype=Yt.dtype) + np.nan
    ho = np.empty(m, dtype=Yh.dtype) + np.nan
    model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + np.nan
    repeat3 = X0.shape[0]
    for g in range(m):
        X0g, Y0g, W0g, X1g, Y1g, W1g, xt, yt, wt, xh, yh, wh = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g],
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk], Yh[:, g], Wh[:, g])
        tcg = np.empty(repeat3, dtype=Y0g.dtype) + np.nan
        for i in range(repeat3):
            x0, y0, w0, x1, y1, w1 = X0g[i], Y0g[i], W0g[i], X1g[i], Y1g[i], W1g[i]
            beta = _lr_fit(x0, y0, w0)
            y1_ = _lr_predict(x1, beta)
            tcg[i] = _lr_rss(y1, y1_, w1)
        tcg.sort()
        tc[g] = tcg.sum()
        beta = _lr_fit(xt, yt, wt)
        model[g] = beta
        yh_ = _lr_predict(xh, beta)
        ho[g] = _lr_rss(yh, yh_, wh)
    tc.sort()
    ho.sort()
    return (tc.sum(), ho.sum()), model

In [5]:
@jit(nopython=True)
def _alg_evaluate_2(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, k, subk):
    tc = np.empty(m, dtype=Yt.dtype) + np.nan
    ho = np.empty(m, dtype=Yh.dtype) + np.nan
    model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + np.nan
    repeat = X0.shape[0]
    for g in range(m):
        X0g, Y0g, W0g, X1g, Y1g, W1g, xt, yt, wt, xh, yh, wh = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g],
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk], Yh[:, g], Wh[:, g])
        tcg = np.empty(repeat * 2, dtype=Y0g.dtype) + np.nan
        for i in range(repeat):
            x0, y0, w0, x1, y1, w1 = X0g[i], Y0g[i], W0g[i], X1g[i], Y1g[i], W1g[i]
            beta0, beta1 = _lr_fit(x0, y0, w0), _lr_fit(x1, y1, w1)
            y1_, y0_ = _lr_predict(x1, beta0), _lr_predict(x0, beta1)
            tcg[2 * i], tcg[2 * i + 1] = _lr_rss(y1, y1_, w1), _lr_rss(y0, y0_, w0)
        tcg.sort()
        tc[g] = tcg.sum()
        beta = _lr_fit(xt, yt, wt)
        model[g] = beta
        yh_ = _lr_predict(xh, beta)
        ho[g] = _lr_rss(yh, yh_, wh)
    tc.sort()
    ho.sort()
    return (tc.sum(), ho.sum()), model

In [6]:
@jit(nopython=True)
def _lr_add_intercept(x): # NOT USED
    return np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

@jit(nopython=True)
def _lr_fit(x, y, w):
    wsqrt = w ** 0.5
    thing = np.linalg.lstsq(np.expand_dims(wsqrt, 1) * x, wsqrt * y)
    return thing[0]

@jit(nopython=True)
def _lr_predict(x, beta):
    return x @ beta.astype(x.dtype)

@jit(nopython=True)
def _lr_rss(y, y_, w):
    res = (y_ - y) ** 2 * w
    res.sort()
    return res.sum()

In [16]:
if __name__ == '__main__':
    o = O()
    def_W(o)
    def_Y(o)
    def_X(o)
    from models.split import split
    s = split2(o.X)
    ta = TheAlg(o.Xa, o.Ya, o.Wa, s)
    ta._setup()
    ta.alg1()

1.0 1.0 0
[] ()

<13> 0
1.0288687417628994 0.9984496220555278 1
[x4] (8,)

<12> 0
1.6107278811722803 1.0470519319703233 2
[x4, x5] (8, 9)

<11> 0
4.280990805481705 1.4171353539242328 3
[x2, x4, x5] (2, 8, 9)

<10> 0
4.841844442439121 2.036917539934384 4
[x2, x4, x5, x6] (2, 8, 9, 10)

<9> 0
5.318702137596149 5.774650711803322 5
[x0, x2, x4, x5, x6] (0, 2, 8, 9, 10)

